<a href="https://colab.research.google.com/github/Denis-Fa/Surpris/blob/main/Surprise_Shir.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install scikit-surprise


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3162653 sha256=22ba750104b1134f67383fb9921ddedaf03fca9b82057b5ca27ea79ffd927973
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


Шаг 1: Импорт необходимых библиотек.
Для начала импортируем необходимые библиотеки, включая Surprise для создания и обучения модели рекомендательной системы.

In [24]:
import random
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

from collections import defaultdict
from surprise import Dataset, SVD

from surprise.model_selection import GridSearchCV


Шаг 2: Создание собственного датасета.
Cоздадим собственный датасет, который будет содержать информацию о пользователях, товарах и их оценках. В этом примере мы создадим случайные данные для пользователей и товаров.

In [25]:
# Создаем список пользователей и инструментов
users = [str(i) for i in range(1, 101)]
tools = [str(i) for i in range(1, 201)]

# Генерируем случайные оценки пользователей для инструментов
data = []
for user in users:
    for tool in tools:
        rating = random.randint(1, 5)
        data.append([user, tool, rating])

# Создаем DataFrame из данных
df = pd.DataFrame(data, columns=['user_id', 'tool_id', 'rating'])

# Создаем объект Reader для определения формата данных
reader = Reader(rating_scale=(1, 5))

# Создаем датасет из DataFrame и объекта Reader
dataset = Dataset.load_from_df(df[['user_id', 'tool_id', 'rating']], reader)

Шаг 3: Разделение данных на обучающий и тестовый наборы.
Для оценки производительности модели разделим данные на обучающий и тестовый наборы.

In [26]:
trainset, testset = train_test_split(dataset, test_size=0.2, random_state=42)


Шаг 4: Создание и обучение модели.
Теперь создадим модель рекомендательной системы. В этом примере мы используем модель SVD.

In [27]:
model = SVD()
model.fit(trainset)

Шаг 5: Получение прогнозов и оценка производительности модели.
Мы получим прогнозы на тестовом наборе и оценим производительность модели.

In [28]:
predictions = model.test(testset)

mae = accuracy.mae(predictions)
rmse = accuracy.rmse(predictions)

print(f'MAE: {mae}')
print(f'RMSE: {rmse}')

MAE:  1.2792
RMSE: 1.4929
MAE: 1.2792415852971248
RMSE: 1.4929336230772479


Шаг 6: Получение рекомендаций для конкретного пользователя.
Теперь мы можем получить персонализированные рекомендации для конкретного пользователя. Для этого выберем случайного пользователя и получим рекомендации.

In [29]:
# Выбираем случайного пользователя
user_id = random.choice(users)

from collections import defaultdict

from surprise import Dataset, SVD


def get_top_n(predictions, n=10):
    """Возвращает первые N рекомендаций для каждого пользователя из набора прогнозов.

     Аргументы:
         предсказания (список объектов предсказаний): список предсказаний, как
             возвращается тестовым методом алгоритма.
         n(int): количество рекомендаций для вывода для каждого пользователя. По умолчанию
             это 10.

     Возврат:
     Словарь, в котором ключи представляют собой идентификаторы пользователей (необработанные), а значения представляют собой списки кортежей:
         [(raw item id, rating estimation), ...] of size n.
    """

    # Сначала сопоставим прогнозы с каждым пользователем..
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Затем отсортируем прогнозы для каждого пользователя и извлечем k самых высоких.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

# Получаем топ N рекомендаций для пользователя
top_n = get_top_n(predictions, n=10)

print(f"Рекомендации инструментов для пользователя {user_id}:")
for tool_id, rating in top_n[user_id]:
    print(f"Инструмент ID: {tool_id}, Рейтинг: {rating}")


Рекомендации инструментов для пользователя 63:
Инструмент ID: 98, Рейтинг: 3.750330925893036
Инструмент ID: 14, Рейтинг: 3.5820111797788385
Инструмент ID: 65, Рейтинг: 3.552820662445498
Инструмент ID: 53, Рейтинг: 3.4700643732860614
Инструмент ID: 19, Рейтинг: 3.4254137210040354
Инструмент ID: 16, Рейтинг: 3.2652963495508196
Инструмент ID: 109, Рейтинг: 3.1282114151994183
Инструмент ID: 69, Рейтинг: 3.086603066231927
Инструмент ID: 25, Рейтинг: 3.015096633850045
Инструмент ID: 50, Рейтинг: 2.972152317106171


Шаг 7: Настройка параметров модели.
Для настройки параметров модели, мы можем использовать методы подбора гиперпараметров, такие как GridSearchCV из scikit-learn.

In [30]:
# Создаем объект модели SVD
model = SVD()

param_grid = {'n_factors': [50, 100, 150],
              'n_epochs': [20, 30, 40],
              'lr_all': [0.002, 0.005, 0.01],
              'reg_all': [0.02, 0.04, 0.06]}

# Используем GridSearchCV для подбора параметров
grid_search = GridSearchCV(SVD,
                           param_grid=param_grid,
                           measures=['rmse'],
                           cv=5)

# Выполняем GridSearchCV
grid_search.fit(dataset)

# Получаем наилучшие параметры
best_params = grid_search.best_params['rmse']
print(f'Наилучшие параметры: {best_params}')

# Создаем объект модели SVD с наилучшими параметрами
model = SVD(n_factors=best_params['n_factors'],
            n_epochs=best_params['n_epochs'],
            lr_all=best_params['lr_all'],
            reg_all=best_params['reg_all'])

# Обучаем модель с наилучшими параметрами
model.fit(trainset)

# Делаем предсказания на тестовом наборе
predictions = model.test(testset)

# Вычисляем показатели производительности
mae = accuracy.mae(predictions)
rmse = accuracy.rmse(predictions)

print(f'MAE: {mae}')
print(f'RMSE: {rmse}')


Наилучшие параметры: {'n_factors': 50, 'n_epochs': 20, 'lr_all': 0.002, 'reg_all': 0.06}
MAE:  1.2316
RMSE: 1.4282
MAE: 1.2316199208891434
RMSE: 1.428208494700999


Шаг 8: Интеграция в магазин инструментов.
Чтобы интегрировать рекомендательную систему в магазин, мы можем создать функцию, которая будет предоставлять рекомендации на основе предпочтений пользователя. Это может быть вызвано на странице пользователя или странице инструмента.

In [31]:
def get_top_n(predictions, user_id, n=10):
    """Получение топ-N рекомендаций для пользователя"""
    top_n = defaultdict(list)
    for uid, iid, _, est, _ in predictions:
        if uid == user_id:
            top_n[uid].append((iid, est))

    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n[user_id]

# Использование функции в магазине
user_id = '42'
recommended_toolss = get_top_n(predictions, user_id)


Для интеграции модели рекомендательной системы с веб-приложением на Java понадобится выполнить следующий шаг:

 Экспортирую обученную модель, созданную в коде Python, в файл (например, в формате pickle), чтобы ее можно было загрузить и использовать в Java. Для этого использую модуль pickle в Python:

In [32]:
import pickle

# Сохранение обученной модели в файл

with open('model_surprise.pkl', 'wb') as file:
    pickle.dump(model, file)



Скачаем на ПК модель